# Imputations and Data Preparations
------

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

np.set_printoptions(suppress=True)

In [198]:
xls = pd.ExcelFile('data/main dataset.xlsx')
ad_post = pd.read_excel(xls, 'Ad-Post')
ad_story = pd.read_excel(xls, 'Ad-Story')
influencer = pd.read_excel(xls, 'Influencer')
leaders_post = pd.read_excel(xls, 'Leaders-Post')
leaders_story = pd.read_excel(xls, 'Leaders-Story')
post = pd.read_excel(xls, 'Post')
story = pd.read_excel(xls, 'Story')
print('Datasets Loaded Completely.')

Datasets Loaded Completely.


In [136]:
null_count_ad_story = len(ad_story[(ad_story['action'].isna()) | (ad_story['impression'].isna())])
null_count_influencer = len(influencer[(influencer['action'].isna()) | (influencer['impression'].isna()) | (influencer['interaction'].isna())])
null_count_leaders_story = len(leaders_story[(leaders_story['impression'].isna())])
null_count_story = len(story[(story['follow'].isna()) | (story['navigation'].isna()) | (story['back'].isna()) | (story['forward'].isna())
                            | (story['next'].isna()) | (story['exit'].isna())])
null_count_leaders_post = len(leaders_post[(leaders_post['view'].isna())])
print(f'Ad-Story Dataset has {null_count_ad_story} records with missing features.')
print(f'Influencer Dataset has {null_count_influencer} records with missing features.')
print(f'Leaders-Story Dataset has {null_count_leaders_story} records with missing features.')
print(f'Story Dataset has {null_count_story} records with missing features.')
print(f'Story Dataset has {null_count_leaders_post} records with missing features.')

Ad-Story Dataset has 17 records with missing features.
Influencer Dataset has 25 records with missing features.
Leaders-Story Dataset has 5 records with missing features.
Story Dataset has 6 records with missing features.
Story Dataset has 1 records with missing features.


## Data Splitting

In this step we need to split the data for imputation. For instance if we want to impute the 'action' feature in ad_story dataset, we must put independent features (in this example, followers, view, interaction and cost) to a new matrix and train the imputer based on it and then predict the missing features with it

In [141]:
ad_story_imputation = ad_story.drop(['ad_story_no', 'name', 'field', 'threshold'], 1)
influencer_imputation = influencer.drop(influencer.columns.difference(['follower', 'view', 'action', 'cost', 'impression', 'cta', 'interaction']), 1)
leaders_story_imputation = leaders_story.drop(leaders_story.columns.difference(['follower', 'view', 'action', 'interaction', 'impression']), 1)
story_imputation = story.drop(['story_no', 'type'], 1)
leaders_post_imputation = leaders_post.drop(['post_no', 'name', 'gender', 'l_threshold', 'h_threshold'], 1)

In [142]:
ad_story_imputation_matrix = ad_story_imputation.values
influencer_imputation_matrix = influencer_imputation.values
leaders_story_imputation_matrix = leaders_story_imputation.values
story_imputation_matrix = story_imputation.values
leaders_post_imputation_matrix = leaders_post_imputation.values

imp_ad_story = IterativeImputer(max_iter = 10)
imp_influencer = IterativeImputer(max_iter = 10)
imp_story = IterativeImputer(max_iter = 10)
imp_leaders_story = IterativeImputer(max_iter = 10)
imp_leaders_post = IterativeImputer(max_iter = 10)

predicted_ad_story = np.round(imp_ad_story.fit_transform(ad_story_imputation_matrix))
predicted_influencer = np.round(imp_influencer.fit_transform(influencer_imputation_matrix))
predicted_story = np.round(imp_story.fit_transform(story_imputation_matrix))
predicted_leaders_story = np.round(imp_leaders_story.fit_transform(leaders_story_imputation_matrix))
predicted_leaders_post = np.round(imp_leaders_post.fit_transform(leaders_post_imputation_matrix))

C:\Users\Ramin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:669: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


In [143]:
df_final_ad_story = pd.DataFrame(predicted_ad_story, columns = ['follower', 'view', 'action', 'interaction', 'impression', 'cost'])
df_final_influencer = pd.DataFrame(predicted_influencer, columns = ['follower', 'view', 'action', 'impression', 'cta', 'interaction', 'cost'])
df_final_story = pd.DataFrame(predicted_story, columns = ['view', 'actions', 'reply', 'profile_visit', 'share', 'website_click', 'sticker_tap', 'impression',
                                                         'follow', 'navigation', 'back', 'forward', 'next', 'exit', 'vote'])
df_final_leaders_story = pd.DataFrame(predicted_leaders_story, columns = ['follower', 'view', 'action', 'interaction', 'impression'])
df_final_leaders_post = pd.DataFrame(predicted_leaders_post, columns = ['follower', 'view', 'like', 'comment', 'share', 'save', 'profile_visit', 'reach', 'impression', 'cost'])

In [199]:
ad_story = ad_story.merge(df_final_ad_story, on='view', how='left')
ad_story.drop(['follower_x', 'action_x', 'interaction_x', 'impression_x', 'cost_x'], axis=1, inplace=True)
ad_story.rename(columns={'follower_y': 'follower',
                         'action_y': 'action',
                         'interaction_y': 'interaction',
                         'impression_y': 'impression',
                         'cost_y': 'cost'},
                inplace=True)
ad_story

,ad_story_no,ad_story_no.1,name,field,view,threshold,follower,action,interaction,impression,cost
0,0,1,4rahesalamat,health,6260,8,686000.0,82.0,7.0,6374.0,190578.0
1,1,2,90tv.official,news,58990,8,877000.0,234.0,90.0,58568.0,444000.0
2,2,3,ancientworld1,fact,101631,8,2600000.0,273.0,218.0,94682.0,556000.0
3,3,4,ayamidooni,fact,97671,8,2300000.0,365.0,488.0,92023.0,650000.0
4,4,5,banooye_khone,women,21887,8,2400000.0,239.0,38.0,74414.0,430000.0
5,5,6,danestani_rooz,fact,205375,8,4500000.0,850.0,523.0,206633.0,1450000.0
6,6,7,diaa_graphy,art & culture,23200,8,311000.0,33.0,116.0,1141.0,104590.0
7,7,8,dialogism,art & culture,12460,8,1100000.0,42.0,30.0,12759.0,75000.0
8,8,9,doctor_khabar,fact,40400,8,1900000.0,135.0,202.0,49339.0,243000.0
9,9,10,filmak20,video,62300,8,1700000.0,188.0,311.0,53222.0,343000.0


In [200]:
influencer = influencer.merge(df_final_influencer, on=df_final_influencer.index , how='left')
influencer.drop(['key_0', 'follower_x', 'view_x', 'action_x', 'impression_x', 'cta_x', 'interaction_x', 'cost_x'], axis=1, inplace=True)
influencer.rename(columns={'follower_y': 'follower',
                         'view_y': 'view',
                         'action_y': 'action',
                         'interaction_y': 'interaction',
                         'impression_y': 'impression',
                         'cta_y': 'cta',
                         'cost_y': 'cost'},
                inplace=True)
influencer

,story_no,story_no.1,influ_name,gender,field,l_threshold,h_threshold,follower,view,action,impression,cta,interaction,cost
0,0,1,ali_bakhtiarvandi,family,lifestyle,20,60,141000.0,3996.0,14.0,4186.0,0.0,0.0,360000.0
1,1,2,ali_bakhtiarvandi,family,lifestyle,20,60,141000.0,3279.0,30.0,3473.0,1.0,28.0,360000.0
2,2,3,ali_bakhtiarvandi,family,lifestyle,20,60,141000.0,3636.0,5.0,3867.0,0.0,0.0,360000.0
3,3,4,ali_bakhtiarvandi,family,lifestyle,20,60,141000.0,3145.0,16.0,3317.0,1.0,11.0,360000.0
4,4,5,ali_bakhtiarvandi,family,lifestyle,20,60,141000.0,3113.0,30.0,3286.0,1.0,22.0,360000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,98,ghazalnevis,female,health,20,60,45100.0,12000.0,229.0,12876.0,1.0,132.0,125000.0
98,98,99,mahshidseydi,family,lifestyle,20,60,89500.0,4854.0,46.0,4945.0,1.0,41.0,625000.0
99,99,100,mahshidseydi,family,lifestyle,20,60,89500.0,4695.0,37.0,4829.0,1.0,35.0,625000.0
100,100,101,mahshidseydi,family,lifestyle,20,60,89500.0,4623.0,44.0,4758.0,1.0,35.0,625000.0


In [201]:
story = story.merge(df_final_story, on=df_final_story.index , how='left')
story.drop(['key_0', 'view_x', 'actions_x', 'reply_x', 'profile_visit_x', 'share_x', 'website_click_x', 'sticker_tap_x', 'impression_x',
            'follow_x', 'navigation_x', 'back_x', 'forward_x', 'next_x', 'exit_x', 'vote_x'],
           axis=1, inplace=True)
story.rename(columns={'view_y': 'view',
                      'actions_y': 'action',
                      'reply_y': 'reply',
                      'profile_visit_y': 'profile_visit',
                      'share_y': 'share',
                      'website_click_y': 'website_click',
                      'sticker_tap_y': 'sticker_tap',
                      'impression_y': 'impression',
                      'follow_y': 'follow',
                      'navigation_y': 'navigation',
                      'back_y': 'back',
                      'forward_y': 'forward',
                      'next_y': 'next',
                      'exit_y': 'exit',
                      'vote_y': 'vote'},
             inplace=True)
story

,story_no,story_no.1,type,view,action,reply,profile_visit,share,website_click,sticker_tap,impression,follow,navigation,back,forward,next,exit,vote
0,0,1,share,1337.0,53.0,4.0,49.0,0.0,0.0,0.0,1380.0,0.0,1618.0,28.0,1048.0,179.0,363.0,0.0
1,1,2,share,1164.0,114.0,2.0,110.0,1.0,1.0,0.0,1190.0,1.0,1490.0,106.0,919.0,119.0,350.0,0.0
2,2,3,share,727.0,21.0,1.0,20.0,0.0,0.0,0.0,765.0,0.0,772.0,38.0,428.0,92.0,214.0,0.0
3,3,4,share,850.0,45.0,5.0,40.0,0.0,0.0,0.0,930.0,1.0,1038.0,31.0,531.0,125.0,351.0,0.0
4,4,5,share,1294.0,69.0,8.0,58.0,0.0,3.0,0.0,1384.0,0.0,1522.0,35.0,909.0,186.0,392.0,0.0
5,5,6,share,1404.0,70.0,3.0,65.0,2.0,0.0,0.0,1465.0,2.0,1702.0,65.0,1160.0,191.0,286.0,0.0
6,6,7,share,1277.0,118.0,6.0,54.0,58.0,0.0,0.0,1316.0,3.0,1649.0,224.0,1106.0,102.0,217.0,0.0
7,7,8,share,1021.0,43.0,3.0,38.0,0.0,2.0,0.0,1097.0,0.0,1372.0,129.0,748.0,107.0,388.0,0.0
8,8,9,share,781.0,26.0,4.0,22.0,0.0,0.0,0.0,806.0,0.0,944.0,14.0,589.0,136.0,205.0,0.0
9,9,10,share,668.0,21.0,3.0,17.0,1.0,0.0,0.0,674.0,1.0,823.0,59.0,505.0,67.0,192.0,0.0


In [202]:
leaders_story = leaders_story.merge(df_final_leaders_story, on=df_final_leaders_story.index , how='left')
leaders_story.drop(['key_0', 'follower_x', 'view_x', 'action_x', 'interaction_x', 'impression_x'],
           axis=1, inplace=True)
leaders_story.rename(columns={'view_y': 'view',
                              'follower_y': 'follower',
                              'action_y': 'action',
                              'interaction_y': 'interaction',
                              'impression_y': 'impression'},
             inplace=True)
leaders_story

,story_no,story_no.1,name,gender,cost,follower,view,action,interaction,impression
0,0,1,aidapooryanasab,female,0,692000.0,103909.0,651.0,562.0,107902.0
1,1,2,alimona.trips,family,0,73400.0,4169.0,162.0,130.0,3548.0
2,2,3,amirparsaneshat,male,0,146000.0,26972.0,527.0,335.0,26925.0
3,3,4,ghonche.ostovarnia,female,0,122000.0,8381.0,205.0,154.0,8381.0
4,4,5,maandani,male,0,128000.0,10493.0,178.0,151.0,10952.0
5,5,6,shahabjafarnejad,male,0,133000.0,7809.0,128.0,74.0,7991.0
6,6,7,yaasamin_,female,0,189000.0,12352.0,288.0,194.0,12640.0
7,7,8,yaasamin_,female,0,189000.0,15021.0,83.0,0.0,13525.0
8,8,9,taaraa.moheb,female,0,757000.0,148197.0,3538.0,2392.0,150001.0
9,9,10,mr.alisaa,family,0,54000.0,5020.0,104.0,59.0,4229.0


In [203]:
leaders_post = leaders_post.merge(df_final_leaders_post, on=df_final_leaders_post.index , how='left')
leaders_post.drop(['key_0', 'follower_x', 'view_x', 'like_x', 'comment_x', 'share_x', 'save_x', 'profile_visit_x', 'reach_x', 'impression_x', 'cost_x'],
           axis=1, inplace=True)
leaders_post.rename(columns={'follower_y': 'follower',
                             'view_y': 'view',
                             'like_y': 'like',
                             'comment_y': 'comment',
                             'share_y': 'share',
                             'save_y': 'save',
                             'profile_visit_y': 'profile_visit',
                             'reach_y': 'reach',
                             'impression_y': 'impression',
                             'cost_y': 'cost'},
                    inplace=True)
leaders_post

,post_no,post_no.1,name,gender,l_threshold,h_threshold,follower,view,like,comment,share,save,profile_visit,reach,impression,cost
0,0,1,aidapooryanasab,female,200,400,692000.0,78137.0,17500.0,205.0,275.0,272.0,1374.0,149048.0,162532.0,30000000.0
1,1,2,alimona.trips,family,200,400,73400.0,20220.0,5099.0,140.0,238.0,138.0,463.0,31642.0,38437.0,5000000.0
2,2,3,amirparsaneshat,male,200,400,146000.0,128378.0,25940.0,573.0,7732.0,7207.0,2593.0,146276.0,180104.0,15200000.0
3,3,4,ghonche.ostovarnia,female,200,400,122000.0,103347.0,12300.0,733.0,261.0,471.0,6611.0,156349.0,172354.0,6000000.0
4,4,5,maandani,male,200,400,128000.0,15002.0,2408.0,68.0,98.0,232.0,482.0,27562.0,30204.0,5200000.0
5,5,6,shahabjafarnejad,male,200,400,133000.0,15701.0,2766.0,35.0,46.0,73.0,125.0,33338.0,36830.0,19000000.0
6,6,7,yaasamin_,female,200,400,189000.0,31714.0,7890.0,211.0,499.0,272.0,427.0,67071.0,74606.0,10000000.0
7,7,8,taaraa.moheb,female,200,400,757000.0,108552.0,12731.0,208.0,207.0,278.0,1060.0,115662.0,171570.0,30000000.0
8,8,9,mr.alisaa,family,200,400,54000.0,6191.0,1201.0,41.0,15.0,24.0,64.0,8311.0,9589.0,2000000.0


In [205]:
book = load_workbook('data/main dataset.xlsx')
writer = pd.ExcelWriter('data/main dataset.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
ad_story.to_excel(writer, "Ad-Story")
influencer.to_excel(writer, "Influencer")
leaders_post.to_excel(writer, "Leaders-Post")
leaders_story.to_excel(writer, "Leaders-Story")
story.to_excel(writer, "Story")
writer.save()

In [197]:
ad_story

,ad_story_no,name,follower,field,view,action,interaction,impression,cost,threshold
0,1,4rahesalamat,686000,health,6260,82.0,7,6374.0,190578,8
1,2,90tv.official,877000,news,58990,234.0,90,58568.0,444000,8
2,3,ancientworld1,2600000,fact,101631,273.0,218,94682.0,556000,8
3,4,ayamidooni,2300000,fact,97671,NaN,488,NaN,650000,8
4,5,banooye_khone,2400000,women,21887,239.0,38,NaN,430000,8
5,6,danestani_rooz,4500000,fact,205375,850.0,523,206633.0,1450000,8
6,7,diaa_graphy,311000,art & culture,23200,NaN,116,NaN,104590,8
7,8,dialogism,1100000,art & culture,12460,42.0,30,12759.0,75000,8
8,9,doctor_khabar,1900000,fact,40400,NaN,202,NaN,243000,8
9,10,filmak20,1700000,video,62300,NaN,311,NaN,343000,8
